# About: Hadoop - Stop the services

----

Hadoop環境の停止用のNotebookです。

停止対象は以下のサービスです。

- ZooKeeper
- HDFS
- YARN
- HBase
- Spark

## *Operation Note*

*This is a cell for your own recording. ここに経緯を記述*

# 操作対象クラスタの設定

停止したい対象のクラスタ名を設定する。

In [1]:
target_group = 'hadoop_all_testcluster'

対象クラスタにAnsibleでpingできることを確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.70 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.112 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.71 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.113 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.114 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


操作のためのPlaybookを準備する。

In [3]:
import os
import tempfile

work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp1_npBA'

In [4]:
!rm -fr {work_dir}/hadoop
!git clone https://github.com/NII-cloud-operation/Literate-computing-Hadoop.git {work_dir}/hadoop
!tree {work_dir}/hadoop

Cloning into '/tmp/tmp1_npBA/hadoop'...
remote: Counting objects: 926, done.
remote: Compressing objects: 100% (281/281), done.
remote: Total 926 (delta 83), reused 0 (delta 0), pack-reused 625
Receiving objects: 100% (926/926), 190.58 KiB | 274.00 KiB/s, done.
Resolving deltas: 100% (229/229), done.
Checking connectivity... done.
/tmp/tmp1_npBA/hadoop
└── playbooks
    ├── conf_base.retry
    ├── conf_base.yml
    ├── conf_hdfs_base.yml
    ├── conf_hdfs_spark.yml
    ├── conf_hdfs_tez.yml
    ├── conf_hdfs_yarn.yml
    ├── conf_namenode_bootstrapstandby.yml
    ├── conf_tez.yml
    ├── enter_hdfs_safemode.yml
    ├── format_namenode.yml
    ├── group_vars
    │   └── all
    │       ├── base
    │       ├── cgroups
    │       ├── collect
    │       ├── f500.dumpall
    │       ├── hbase_master
    │       ├── hbase_regionserver
    │       ├── hcatalog
    │       ├── hdfs_base
    │       ├── hdfs_spark
    │       ├── hdfs_tez
    │       ├── hdfs_yarn
    │       ├── hive
    │ 

In [5]:
playbook_dir = os.path.join(work_dir, 'hadoop/playbooks')
!ls -la {playbook_dir} | head

total 244
drwxr-xr-x  4 root root 4096 Aug 19 07:18 .
drwxr-xr-x  4 root root 4096 Aug 19 07:18 ..
-rw-r--r--  1 root root   13 Aug 19 07:18 conf_base.retry
-rw-r--r--  1 root root   39 Aug 19 07:18 conf_base.yml
-rw-r--r--  1 root root  136 Aug 19 07:18 conf_hdfs_base.yml
-rw-r--r--  1 root root  137 Aug 19 07:18 conf_hdfs_spark.yml
-rw-r--r--  1 root root  135 Aug 19 07:18 conf_hdfs_tez.yml
-rw-r--r--  1 root root  136 Aug 19 07:18 conf_hdfs_yarn.yml
-rw-r--r--  1 root root  188 Aug 19 07:18 conf_namenode_bootstrapstandby.yml


これでPlaybookの準備はOK。

# Sparkの停止

Spark HistoryServerを停止する。

## Spark HistoryServerの停止

In [6]:
!ansible-playbook -l {target_group} {playbook_dir}/stop_spark_historyserver.yml


PLAY [hadoop_spark_history] ****************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]

TASK [check_status_spark_history_server] ***************************************
ok: [XXX.XXX.XXX.72]

TASK [stop_spark_history_server] ***********************************************
changed: [XXX.XXX.XXX.72]

TASK [delete_pid_file] *********************************************************
ok: [XXX.XXX.XXX.72]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.72               : ok=4    changed=1    unreachable=0    failed=0   



# HBaseの停止

Master, RegionServerを停止する。


## RegionServerの停止

In [7]:
!ansible-playbook {playbook_dir}/stop_hbase_regionserver.yml -l {target_group}


PLAY [hadoop_hbase_regionserver] ***********************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.114]

TASK [stop_hbase-regionserver] *************************************************
changed: [XXX.XXX.XXX.112]
changed: [XXX.XXX.XXX.113]
changed: [XXX.XXX.XXX.114]
changed: [XXX.XXX.XXX.73]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.112              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.113              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.73               : ok=2    changed=1    unreachable=0    failed=0   



## Masterの停止

In [8]:
!ansible-playbook {playbook_dir}/stop_hbase_master.yml -l {target_group}


PLAY [hadoop_hbase_master] *****************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]

TASK [stop_hbase-master] *******************************************************
changed: [XXX.XXX.XXX.70]
changed: [XXX.XXX.XXX.71]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.70               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.71               : ok=2    changed=1    unreachable=0    failed=0   



# YARNの停止

ResourceManager, NodeManager, MapReduce HistoryServerを停止する。

## MapReduce HistoryServerの停止

In [9]:
!ansible-playbook {playbook_dir}/stop_mapreduce_historyserver.yml -l {target_group}


PLAY [hadoop_mapreduce_historyserver] ******************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]

TASK [stop_mapreduce-historyserver] ********************************************
changed: [XXX.XXX.XXX.72]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.72               : ok=2    changed=1    unreachable=0    failed=0   



## YARN TimelineServiceの停止

In [10]:
!ansible-playbook {playbook_dir}/stop_timelineservice.yml -l {target_group}


PLAY [hadoop_timelineservice] **************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]

TASK [stop_hadoop-yarn-timelineserver] *****************************************
changed: [XXX.XXX.XXX.72]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.72               : ok=2    changed=1    unreachable=0    failed=0   



## NodeManagerの停止

In [11]:
!ansible-playbook {playbook_dir}/stop_nodemanager.yml -l {target_group}


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.114]

TASK [stop_hadoop-yarn-nodemanager] ********************************************
changed: [XXX.XXX.XXX.112]
changed: [XXX.XXX.XXX.73]
changed: [XXX.XXX.XXX.113]
changed: [XXX.XXX.XXX.114]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.112              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.113              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.73               : ok=2    changed=1    unreachable=0    failed=0   



## ResourceManagerの停止

In [12]:
!ansible-playbook {playbook_dir}/stop_resourcemanager.yml -l {target_group}


PLAY [hadoop_resourcemanager] **************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.70]

TASK [stop_hadoop-yarn-resourcemanager] ****************************************
changed: [XXX.XXX.XXX.70]
changed: [XXX.XXX.XXX.71]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.70               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.71               : ok=2    changed=1    unreachable=0    failed=0   



# HDFSの停止

NameNode, DataNode, JournalNodeを停止する。

## 現在の状態を確認

In [13]:
haadmin_stdout = !ansible hadoop_namenode -l {target_group} -s -U hdfs -m shell -a 'timeout 15 hdfs haadmin -getServiceState $(hostname)'
haadmin_result = [line.split()[0] for line in haadmin_stdout if len(line) > 0]
active_namenode_host = haadmin_result[haadmin_result.index("active") - 1]
standby_namenode_host = haadmin_result[haadmin_result.index("standby") - 1]

print("active_namenode_host = '%s'" % active_namenode_host)
print("standby_namenode_host = '%s'" % standby_namenode_host)

active_namenode_host = 'XXX.XXX.XXX.70'
standby_namenode_host = 'XXX.XXX.XXX.71'


## HDFSをセーフモードにする

In [14]:
!ansible-playbook {playbook_dir}/enter_hdfs_safemode.yml -l { standby_namenode_host }


PLAY [hadoop_namenode] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.71]

TASK [HDFS_enter_safemode] *****************************************************
changed: [XXX.XXX.XXX.71]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.71               : ok=2    changed=1    unreachable=0    failed=0   



## NameNodeクラスタの停止

まずStandBy側のZKFC/NameNodeを停止させる

In [15]:
!ansible-playbook {playbook_dir}/stop_namenode.yml -l { standby_namenode_host }


PLAY [hadoop_namenode] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.71]

TASK [stop_hadoop-hdfs-zkfc] ***************************************************
changed: [XXX.XXX.XXX.71]

TASK [stop_hadoop-hdfs-namenode] ***********************************************
changed: [XXX.XXX.XXX.71]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.71               : ok=3    changed=2    unreachable=0    failed=0   



Active側のZKFC/NameNodeを停止させる

In [16]:
!ansible-playbook {playbook_dir}/stop_namenode.yml -l { active_namenode_host }


PLAY [hadoop_namenode] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.70]

TASK [stop_hadoop-hdfs-zkfc] ***************************************************
changed: [XXX.XXX.XXX.70]

TASK [stop_hadoop-hdfs-namenode] ***********************************************
changed: [XXX.XXX.XXX.70]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.70               : ok=3    changed=2    unreachable=0    failed=0   



## DataNodeの停止

In [17]:
!ansible-playbook {playbook_dir}/stop_datanode.yml -l {target_group}


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.114]

TASK [stop_hadoop-hdfs-datanode] ***********************************************
changed: [XXX.XXX.XXX.113]
changed: [XXX.XXX.XXX.112]
changed: [XXX.XXX.XXX.73]
changed: [XXX.XXX.XXX.114]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.112              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.113              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.114              : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.73               : ok=2    changed=1    unreachable=0    failed=0   



## JournalNodeの停止

In [18]:
!ansible-playbook {playbook_dir}/stop_journalnode.yml -l {target_group}


PLAY [hadoop_journalnode] ******************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.72]

TASK [stop_hadoop-hdfs-journalnode] ********************************************
changed: [XXX.XXX.XXX.70]
changed: [XXX.XXX.XXX.71]
changed: [XXX.XXX.XXX.72]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.70               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.71               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.72               : ok=2    changed=1    unreachable=0    failed=0   



# ZooKeeperの停止

In [19]:
!ansible-playbook {playbook_dir}/stop_zookeeper-server.yml -l {target_group}


PLAY [hadoop_zookeeperserver] **************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.72]

TASK [stop_zookeeper-server] ***************************************************
changed: [XXX.XXX.XXX.70]
changed: [XXX.XXX.XXX.72]
changed: [XXX.XXX.XXX.71]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.70               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.71               : ok=2    changed=1    unreachable=0    failed=0   
XXX.XXX.XXX.72               : ok=2    changed=1    unreachable=0    failed=0   



# 後始末

一時ファイルの後始末・・・

In [20]:
!rm -fr {work_dir}